In [1]:
# Dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [2]:
df = pd.read_excel('Final Project\ProjectData.xlsx')
df.head()

,Store Number,Activations,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Profitability
0,5278,476,1020,30571,43.4,51255,27207,3163,10.346407,Profitable
1,2683,317,1035,5254,45.0,73250,36471,403,7.670346,Profitable
2,2901,207,1060,15026,34.2,52607,33655,2205,14.674564,Not Profitable
3,2386,407,1082,10318,45.3,51717,28354,1372,13.297151,Profitable
4,2174,357,1085,40721,39.1,59881,28943,3690,9.061664,Profitable


In [3]:
df.count()

Store Number         1167
Activations          1167
Zipcode              1167
Population           1167
Median Age           1167
Household Income     1167
Per Capita Income    1167
Poverty Count        1167
Poverty Rate         1167
Profitability        1167
dtype: int64

In [4]:
df.columns

Index(['Store Number', 'Activations', 'Zipcode', 'Population', 'Median Age',
       'Household Income', 'Per Capita Income', 'Poverty Count',
       'Poverty Rate', 'Profitability'],
      dtype='object')

In [5]:
df = df[['Activations', 'Zipcode', 'Population',
       'Median Age', 'Household Income', 'Per Capita Income', 'Poverty Count',
       'Poverty Rate', 'Profitability']]
df.head()

,Activations,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Profitability
0,476,1020,30571,43.4,51255,27207,3163,10.346407,Profitable
1,317,1035,5254,45.0,73250,36471,403,7.670346,Profitable
2,207,1060,15026,34.2,52607,33655,2205,14.674564,Not Profitable
3,407,1082,10318,45.3,51717,28354,1372,13.297151,Profitable
4,357,1085,40721,39.1,59881,28943,3690,9.061664,Profitable


In [6]:
X = df.drop("Profitability", axis=1)
y =df["Profitability"]
print(X.shape, y.shape)

(1167, 8) (1167,)


In [7]:
# label encoding and one-hot encoding

label_encoder_Zipcode = LabelEncoder()
label_encoder_Zipcode.fit(X["Zipcode"])
encoded_Zipcode = label_encoder_Zipcode.transform(X["Zipcode"])

Zipcode_categorical = to_categorical(encoded_Zipcode)
Zipcode_categorical.shape

(1167, 1167)

In [8]:
#Converting categorical data to a dataframe

Zipcode_categorical_df = pd.DataFrame(Zipcode_categorical)

In [9]:
#Concatinating other X features with categorical zipcode
X_exZipcode_df = X[['Activations', 'Population','Median Age', 'Household Income', 'Per Capita Income', 'Poverty Count',
       'Poverty Rate']]
X_df = pd.concat([X_exZipcode_df,Zipcode_categorical_df],axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, random_state=1, stratify=y)

In [11]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Riad\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype float32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [12]:
# label encoding and one-hot encoding

label_encoder_y = LabelEncoder()
label_encoder_y.fit(y_train)
encoded_y_train = label_encoder_y.transform(y_train)
encoded_y_test = label_encoder_y.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [13]:
#import dependecies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=1174))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [15]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               117500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 127,802
Trainable params: 127,802
Non-trainable params: 0
_________________________________________________________________


In [17]:
X_train_scaled.shape

(875, 1174)

In [18]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=50,
    shuffle=True,
    verbose=0
)

In [19]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Profitability Prediction - Loss: {model_loss}, Accuracy: {model_accuracy}")

292/1 - 0s - loss: 0.0630 - accuracy: 0.9555
Profitability Prediction - Loss: 0.11052568208375206, Accuracy: 0.9554794430732727


In [20]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder_y.inverse_transform(encoded_predictions)

In [21]:
Result_df = pd.DataFrame({'Predicted Class':prediction_labels,
                        'Actual Labels':(y_test[:10])})
Result_df

,Predicted Class,Actual Labels
822,Not Profitable,Not Profitable
380,Profitable,Profitable
791,Profitable,Not Profitable
879,Not Profitable,Not Profitable
165,Profitable,Profitable
660,Not Profitable,Not Profitable
768,Profitable,Profitable
649,Profitable,Profitable
503,Not Profitable,Not Profitable
924,Not Profitable,Not Profitable


In [22]:
Result_df.to_excel('Final Project\Validation.xlsx',index=False)

## Predicting with External Data

In [23]:
data_df = pd.read_excel('Final Project\PredictData.xlsx')
data_df.head()

,Activations,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,309,601,18450,36.6,12041.0,7380.0,10816,58.623306
1,83,602,41302,38.6,15663.0,8463.0,22409,54.256452
2,106,603,53683,38.9,15485.0,9176.0,26220,48.842278
3,113,606,6591,37.3,15019.0,6383.0,3721,56.455773
4,27,610,28963,39.2,16707.0,7892.0,14569,50.302110


In [24]:
data_df.count()

Activations          31953
Zipcode              31953
Population           31953
Median Age           31422
Household Income     30818
Per Capita Income    31379
Poverty Count        31953
Poverty Rate         31617
dtype: int64

In [25]:
data_df  = data_df.dropna(how='any')

In [26]:
data_df.count()

Activations          30818
Zipcode              30818
Population           30818
Median Age           30818
Household Income     30818
Per Capita Income    30818
Poverty Count        30818
Poverty Rate         30818
dtype: int64

In [27]:
data_df = data_df.iloc[:1167]
data_df.head()

,Activations,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,309,601,18450,36.6,12041.0,7380.0,10816,58.623306
1,83,602,41302,38.6,15663.0,8463.0,22409,54.256452
2,106,603,53683,38.9,15485.0,9176.0,26220,48.842278
3,113,606,6591,37.3,15019.0,6383.0,3721,56.455773
4,27,610,28963,39.2,16707.0,7892.0,14569,50.302110


In [28]:
data_df.shape

(1167, 8)

In [29]:
label_encoder_zcode = LabelEncoder()
label_encoder_zcode.fit(data_df["Zipcode"])
encoded_zcode = label_encoder_zcode.transform(data_df["Zipcode"])

zcode_categorical = to_categorical(encoded_zcode)
zcode_categorical.shape

(1167, 1167)

In [30]:
zcode_categorical_df = pd.DataFrame(zcode_categorical)
data_exZipcode_df = data_df[['Activations', 'Population','Median Age', 'Household Income', 'Per Capita Income', 'Poverty Count',
       'Poverty Rate']]
input_df = pd.concat([data_exZipcode_df,zcode_categorical_df],axis=1)
input_df.head()

,Activations,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,0,1,2,...,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166
0,309.0,18450.0,36.6,12041.0,7380.0,10816.0,58.623306,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,83.0,41302.0,38.6,15663.0,8463.0,22409.0,54.256452,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,106.0,53683.0,38.9,15485.0,9176.0,26220.0,48.842278,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,113.0,6591.0,37.3,15019.0,6383.0,3721.0,56.455773,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,27.0,28963.0,39.2,16707.0,7892.0,14569.0,50.302110,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
input_scaler = MinMaxScaler().fit(input_df)
input_train_scaled = input_scaler.transform(input_df)

C:\Users\Riad\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [32]:
encoded_predictions_data = model.predict_classes(input_train_scaled[0:1167])
prediction_labels_output = label_encoder_y.inverse_transform(encoded_predictions_data)

In [33]:
predict_df = pd.DataFrame({'Predicted Class':prediction_labels_output})
                        
output_with_prediction_df = pd.concat([data_df,predict_df],axis = 1)
output_with_prediction_df.head()

,Activations,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Predicted Class
0,309.0,601.0,18450.0,36.6,12041.0,7380.0,10816.0,58.623306,Profitable
1,83.0,602.0,41302.0,38.6,15663.0,8463.0,22409.0,54.256452,Profitable
2,106.0,603.0,53683.0,38.9,15485.0,9176.0,26220.0,48.842278,Not Profitable
3,113.0,606.0,6591.0,37.3,15019.0,6383.0,3721.0,56.455773,Profitable
4,27.0,610.0,28963.0,39.2,16707.0,7892.0,14569.0,50.302110,Profitable


In [34]:
output_with_prediction_df.to_excel('Final Project\FinalOutput.xlsx',index=False)

In [35]:
predict_df.to_excel('Final Project\Prediction.xlsx',index=False)